In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr'

%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/mink/notebooks/CameraTraps')  # append this repo to PYTHONPATH
sys.path.append('/home/mink/lib/ai4eutils')

In [3]:
import json
import os
from collections import Counter, defaultdict
from random import sample
from shutil import copyfile
from multiprocessing.pool import ThreadPool

import numpy as np
import pandas as pd
from tqdm import tqdm

import path_utils  # ai4eutils

from data_management.megadb.schema import sequences_schema_check
from data_management.megadb.converters.cct_to_megadb import process_sequences

# rspb_atkinson

In [4]:
dataset_name = 'rspb_atkinson'

container_root = '/mink_disk_0/camtraps/rspb_atkinson/rspb-atkinson/'  

path_to_output = f'/home/mink/camtraps/data/megadb_jsons/{dataset_name}.json' 
path_to_output_temp = f'/home/mink/camtraps/data/megadb_jsons/{dataset_name}_temp.json' 

## Step 0 - Add an entry to the `datasets` table

Done

## Step 1 - Prepare the `sequence` objects to insert into the database

No sequence info and no timestamp extracted. 

Most have location info.

In [5]:
# Small Test folder
sequences_test = []

for fn in os.listdir(os.path.join(container_root, 'Test')):
    sequences_test.append({
        'dataset': dataset_name,
        'seq_id': f'dummy_Test_{len(sequences_test)}',
        'images': [
            {
                'file': os.path.join('Test', fn),
                'frame_num': 1 # only one image, but easier for ingesting the annotations
            }
        ],
        'location': 'Test',
        'class': ['animal']
    })
len(sequences_test)
sequences_test[0]

4

{'dataset': 'rspb_atkinson',
 'seq_id': 'dummy_Test_0',
 'images': [{'file': 'Test/vlcsnap-error200.png', 'frame_num': 1}],
 'location': 'Test',
 'class': ['animal']}

In [6]:
paths = path_utils.recursive_file_list(os.path.join(container_root, 'AI sample'))
paths = [p for p in paths if path_utils.is_image_file(p)]

len(paths)
paths[-1]

9791

'/mink_disk_0/camtraps/rspb_atkinson/rspb-atkinson/AI sample/OP205/ONWP364/IMG_0018.JPG'

In [7]:
sequences_2 = []

for p in paths:
    
    location = '/'.join(p.split('/')[-3:-1])
    
    sequences_2.append({
        'dataset': dataset_name,
        'seq_id': f'dummy_sample_{len(sequences_2)}',
        'location': location,
        'class': ['tbd'],
        'images': [
            {
                'file': p.split(container_root)[1],
                'frame_num': 1
            }
        ]
    })
    
len(sequences_2)
sequences_2[100]

9791

{'dataset': 'rspb_atkinson',
 'seq_id': 'dummy_sample_100',
 'location': 'OP211/ONWP335',
 'class': ['tbd'],
 'images': [{'file': 'AI sample/OP211/ONWP335/IMG_0256.JPG', 'frame_num': 1}]}

In [8]:
sequences_2[-1]

{'dataset': 'rspb_atkinson',
 'seq_id': 'dummy_sample_9790',
 'location': 'OP205/ONWP364',
 'class': ['tbd'],
 'images': [{'file': 'AI sample/OP205/ONWP364/IMG_0018.JPG', 'frame_num': 1}]}

In [9]:
sequences_test.extend(sequences_2)

In [10]:
sequences = sequences_test
len(sequences)

9795

## Step 2 - Pass the schema check

Once your metadata are in the MegaDB format for `sequence` items, we check that they conform to the format's schema.

If the format conforms, the following messages will be printed:

```
Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.
```

For large datasets, the second step will take some time (~ a minute). 

Otherwise there will be an error message describing what's wrong. Please fix the issues until all checks are passed. You might need to write some snippets of code to loop through the `sequence` items to understand which entries have problems.

In [12]:
%%time

sequences_schema_check.sequences_schema_check(sequences)

Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.
CPU times: user 1.15 s, sys: 11.2 ms, total: 1.16 s
Wall time: 1.16 s


In [13]:
with open(path_to_output_temp, 'w', encoding='utf-8') as f:
    json.dump(sequences, f, indent=1, ensure_ascii=False)

### Step 2b - copy images to flat folder

In [14]:
def copy_file(src_path, dst_path):
    return copyfile(src_path, dst_path)

In [15]:
%%time

path_pairs = []
for seq in tqdm(sequences):
    seq_id = seq['seq_id']
    for im in seq['images']:
        src_path = os.path.join(container_root, im['file'])
        assert os.path.exists(src_path), src_path
        frame = 1
        dst_path = os.path.join('/mink_disk_0/camtraps/imerit12b', 
                                f'{dataset_name}.seq{seq_id}.frame{frame}.jpg')
        path_pairs.append((src_path, dst_path))

100%|██████████| 9795/9795 [00:00<00:00, 119143.58it/s]

CPU times: user 69.8 ms, sys: 16 ms, total: 85.7 ms
Wall time: 84.3 ms


In [16]:
len(path_pairs)  # non-empty images out of total of 19221 (38%)
path_pairs[-100]

9795

('/mink_disk_0/camtraps/rspb_atkinson/rspb-atkinson/AI sample/OP207/ONWP334/IMG_0247.JPG',
 '/mink_disk_0/camtraps/imerit12b/rspb_atkinson.seqdummy_sample_9691.frame1.jpg')

In [17]:
%%time

with ThreadPool(8) as pool:
    dst_paths = pool.starmap(copy_file, path_pairs)

CPU times: user 29.7 s, sys: 1min 22s, total: 1min 51s
Wall time: 2min 1s


In [18]:
len(dst_paths)

9795